In [43]:
import pandas as pd


In [44]:
data = pd.read_hdf('data_feat.h5', 'table')

In [45]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month', 'date_block_num'], axis=1).fillna(-999)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month', 'date_block_num'], axis=1).fillna(-999)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month', 'date_block_num'], axis=1).fillna(-999)

In [46]:
Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

In [47]:
import numpy as np
# catboost would give error with float16 
float_cols = [c for c in X_train if X_train[c].dtype == "float16"]
X_train[float_cols] = X_train[float_cols].astype(np.float32)
X_valid[float_cols] = X_valid[float_cols].astype(np.float32)
X_test[float_cols] = X_test[float_cols].astype(np.float32)

int_cols =   [c for c in X_train if X_train[c].dtype == "int8"]
X_train[int_cols] = X_train[int_cols].astype(np.int16)
X_valid[int_cols] = X_valid[int_cols].astype(np.int16)
X_test[int_cols] = X_test[int_cols].astype(np.int16)

Y_train = Y_train.astype(np.float32)
Y_valid = Y_valid.astype(np.float32)


In [ ]:
# grid search parameters
from catboost import CatBoostRegressor
import time

# ts = time.time()

# grid = {'learning_rate': [0.01, 0.1],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9],
#         'bagging_temperature':[0.2, 0.4, 0.6, 0.8]}

# from sklearn.model_selection import ParameterGrid
# res = []
# for param in ParameterGrid(grid):
#     model = CatBoostRegressor(iterations=1000,
#                           subsample=0.7,
#                           random_seed = 44,
#                           task_type="GPU",
#                           verbose=True,
#                           od_type='Iter',
#                           od_wait=30,
#                           **param
#                          )
#     model.fit(X_train, Y_train, 
#              eval_set=(X_valid, Y_valid)
#              )
#     res.append((model.best_score_, param))
    


# model.fit(
#     X_train, Y_train,
#     #cat_features=cat_features,
#     eval_set=(X_valid, Y_valid),
#     plot=True
# )

# print(time.time() - ts)

In [23]:
# for perf, param in res:
#     if perf['learn']['RMSE'] < 0.84:
#         print(param)
#         print('----')

{'bagging_temperature': 0.2, 'depth': 10, 'l2_leaf_reg': 3, 'learning_rate': 0.1}
----
{'bagging_temperature': 0.2, 'depth': 10, 'l2_leaf_reg': 9, 'learning_rate': 0.1}
----
{'bagging_temperature': 0.4, 'depth': 10, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
----
{'bagging_temperature': 0.4, 'depth': 10, 'l2_leaf_reg': 7, 'learning_rate': 0.1}
----
{'bagging_temperature': 0.4, 'depth': 10, 'l2_leaf_reg': 9, 'learning_rate': 0.1}
----
{'bagging_temperature': 0.6, 'depth': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
----
{'bagging_temperature': 0.8, 'depth': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
----
{'bagging_temperature': 0.8, 'depth': 10, 'l2_leaf_reg': 9, 'learning_rate': 0.1}
----


In [41]:
model = CatBoostRegressor(iterations=1000,
                          subsample=0.8,
                          random_seed = 44,
                          task_type="GPU",
                          verbose=True,
                          od_type='Iter',
                          od_wait=30,
                          depth=8,
                          learning_rate=0.1,
                          # l2_leaf_reg=1,
                          bootstrap_type='Poisson',
                          grow_policy='Lossguide'
                         )
model.fit(X_train, Y_train, 
          eval_set=(X_valid, Y_valid)
         )

0:	learn: 1.1674602	test: 1.1017695	best: 1.1017695 (0)	total: 139ms	remaining: 2m 19s
1:	learn: 1.1258956	test: 1.0709323	best: 1.0709323 (1)	total: 264ms	remaining: 2m 11s
2:	learn: 1.0908401	test: 1.0457676	best: 1.0457676 (2)	total: 351ms	remaining: 1m 56s
3:	learn: 1.0610372	test: 1.0257515	best: 1.0257515 (3)	total: 440ms	remaining: 1m 49s
4:	learn: 1.0358793	test: 1.0092167	best: 1.0092167 (4)	total: 520ms	remaining: 1m 43s
5:	learn: 1.0149240	test: 0.9948012	best: 0.9948012 (5)	total: 605ms	remaining: 1m 40s
6:	learn: 0.9969969	test: 0.9830091	best: 0.9830091 (6)	total: 688ms	remaining: 1m 37s
7:	learn: 0.9811616	test: 0.9712520	best: 0.9712520 (7)	total: 771ms	remaining: 1m 35s
8:	learn: 0.9678378	test: 0.9616287	best: 0.9616287 (8)	total: 852ms	remaining: 1m 33s
9:	learn: 0.9563749	test: 0.9534381	best: 0.9534381 (9)	total: 937ms	remaining: 1m 32s
10:	learn: 0.9465547	test: 0.9483042	best: 0.9483042 (10)	total: 1.02s	remaining: 1m 31s
11:	learn: 0.9382160	test: 0.9429352	best

In [ ]:
# save the catboost model
model.save_model('catboost_model',
           format="cbm",
           export_parameters=None,
           pool=None)

In [50]:
# load the mode
from catboost import CatBoostRegressor

from_file = CatBoostRegressor()
from_file.load_model("catboost_20200809")

In [51]:
# prepare submission

test = pd.read_csv("test.csv")
Y_test = from_file.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('catboost_submission.csv', index=False)